# Logistic Regression
---
This notebook uses Cirrus to run logistic regression on the Criteo dataset.

## Setup
---

In [1]:
# To ease development, each time a cell is run, all modules will be reloaded.
%load_ext autoreload
%autoreload 2

In [2]:
import logging
import sys
import atexit

In [3]:
# Cirrus produces logs, but they will not show unless we add a handler that prints.
from cirrus import utilities
utilities.set_logging_handler()

/home/tkice/anaconda3/envs/cirrus_env_2.7/lib/python2.7/site-packages/cryptography/__init__.py:39: CryptographyDeprecationWarning: Python 2 is no longer supported by the Python core team. Support for it is now deprecated in cryptography, and will be removed in a future release.
  CryptographyDeprecationWarning,


[     _initialize | ResourceManager] Initializing Lambda client.
[     _initialize | ResourceManager] Initializing no-retries Lambda client.
[     _initialize | ResourceManager] Initializing IAM resource.
[     _initialize | ResourceManager] Initializing EC2 resource.
[     _initialize | ResourceManager] Initializing Cloudwatch Logs client.
[     _initialize | ResourceManager] Initializing S3 resource.
[     _initialize | ResourceManager] Initializing STS client.


In [4]:
from cirrus import instance, parameter_server, automate, lr

## Instance, server, and task
---

First, we start an EC2 instance.

In [5]:
inst = instance.Instance(
    name="lr_example_instance",
    disk_size=32,
    typ="m4.2xlarge",
    username="ubuntu",
    ami_owner_name=("self", "cirrus_server_image")
)
inst.start()

[        __init__ |      MainThread] Resolving AMI owner/name to AMI ID.
[        __init__ |      MainThread] Done.
[         _exists |      MainThread] Listing instances.
[         _exists |      MainThread] No existing instance with the same name was found.
[ _start_and_wait |      MainThread] Starting a new instance.
[ _start_and_wait |      MainThread] Waiting for instance to enter running state.
[ _start_and_wait |      MainThread] Fetching instance metadata.
[ _start_and_wait |      MainThread] Done.
[           start |      MainThread] Done.


Second, we create a parameter server to run on our instance.

In [6]:
server = parameter_server.ParameterServer(
    instance=inst,
    ps_port=1337,
    error_port=1338,
    num_workers=64
)

Third, we define our machine learning task.

In [7]:
task = lr.LogisticRegression(
    n_workers=10,
    n_ps=1,
    dataset="criteo-kaggle-19b",
    learning_rate=0.0001,
    epsilon=0.0001,
    progress_callback=None,
    train_set=(0, 799),
    test_set=(800, 850),
    minibatch_size=200,
    model_bits=19,
    ps=server,
    opt_method="adagrad",
    timeout=60,
    lambda_size=192
)

## Run
---

Next, we run our machine learning task.

In [8]:
task.run()

[           start |      MainThread] Uploading configuration.
[     run_command |      MainThread] Calling _connect_ssh.
[    _connect_ssh |      MainThread] Configuring.
[    _connect_ssh |      MainThread] Making connection attempt #1 out of 35.


/home/tkice/anaconda3/envs/cirrus_env_2.7/lib/python2.7/site-packages/paramiko/kex_ecdh_nist.py:39: CryptographyDeprecationWarning:

encode_point has been deprecated on EllipticCurvePublicNumbers and will be removed in a future version. Please use EllipticCurvePublicKey.public_bytes to obtain both compressed and uncompressed point encoding.

/home/tkice/anaconda3/envs/cirrus_env_2.7/lib/python2.7/site-packages/paramiko/kex_ecdh_nist.py:96: CryptographyDeprecationWarning:

Support for unsafe construction of public numbers from encoded data will be removed in a future version. Please use EllipticCurvePublicKey.from_encoded_point

/home/tkice/anaconda3/envs/cirrus_env_2.7/lib/python2.7/site-packages/paramiko/kex_ecdh_nist.py:111: CryptographyDeprecationWarning:

encode_point has been deprecated on EllipticCurvePublicNumbers and will be removed in a future version. Please use EllipticCurvePublicKey.public_bytes to obtain both compressed and uncompressed point encoding.



[     run_command |      MainThread] Running `echo 'load_input_path: /mnt/efs/criteo_kaggle/train.csv 
load_input_type: csv
dataset_format: binary
num_classes: 2 
num_features: 13 
limit_cols: 14 
normalize: 0 
limit_samples: 10000 
s3_size: 50000 
use_bias: 1 
model_type: LogisticRegression 
minibatch_size: 200 
learning_rate: 0.000100 
epsilon: 0.000100 
model_bits: 19 
s3_bucket: criteo-kaggle-19b 
use_grad_threshold: 0 
grad_threshold: 0.001000 
train_set: 0-799 
test_set: 800-850' > config_1337.txt`.
[     run_command |      MainThread] Waiting for completion.
[     run_command |      MainThread] Fetching stdout and stderr.
[     run_command |      MainThread] stdout had length 0.
[     run_command |      MainThread] stderr had length 0.
[     run_command |      MainThread] Exit code was 0.
[     run_command |      MainThread] Done.
[           start |      MainThread] Starting parameter server.
[     run_command |      MainThread] Running `ulimit -c unlimited; nohup ./parameter_s

Run this cell to see the present accuracy of the model.

In [9]:
for line in server.error_output().split("\n")[-10:]:
    if "Accuracy" in line:
        print(line)

[     run_command |      MainThread] Running `cat error_out_1337`.
[     run_command |      MainThread] Waiting for completion.
[     run_command |      MainThread] Fetching stdout and stderr.
[   launch_worker | Exp #00 Wkr #03] Task 30001 completed with status code 200.
[   launch_worker | Exp #00 Wkr #09] Task 90001 completed with status code 200.
[   launch_worker | Exp #00 Wkr #04] Task 40001 completed with status code 200.
[   launch_worker | Exp #00 Wkr #00] Task 1 completed with status code 200.
[     run_command |      MainThread] stdout had length 13084.
[     run_command |      MainThread] stderr had length 0.
[     run_command |      MainThread] Exit code was 0.
[     run_command |      MainThread] Done.
[ERROR_TASK] Loss (Total/Avg): 1.7328e+06/0.693122 Accuracy: 0.700458 time(us): 1597050917310826 time from start (sec): 15.3184
[ERROR_TASK] Loss (Total/Avg): 1.7328e+06/0.693122 Accuracy: 0.700458 time(us): 1597050919924366 time from start (sec): 17.932
[   launch_worker |

## Cleanup
---

When we're satisfied with the results, we kill our task.

In [10]:
task.kill()

[     run_command |      MainThread] Running `kill -n 9 $(cat error_1337.pid)`.
[   delete_lambda | Exp #00 Cleanup] Deleting Lambda function cirrus_worker_0_2020-8-10_2-14-49-268737.
[     run_command |      MainThread] Waiting for completion.
[     run_command |      MainThread] Fetching stdout and stderr.
[     run_command |      MainThread] stdout had length 0.
[     run_command |      MainThread] stderr had length 0.
[     run_command |      MainThread] Exit code was 0.
[     run_command |      MainThread] Done.
[     run_command |      MainThread] Running `kill -n 9 $(cat ps_1337.pid)`.
[     run_command |      MainThread] Waiting for completion.
[     run_command |      MainThread] Fetching stdout and stderr.
[     run_command |      MainThread] stdout had length 0.
[     run_command |      MainThread] stderr had length 0.
[     run_command |      MainThread] Exit code was 0.
[     run_command |      MainThread] Done.


We also need to terminate our instance in order to avoid continuing charges.

In [11]:
inst.cleanup()

[         cleanup |      MainThread] Closing SSH client.
[         cleanup |      MainThread] Terminating instance.
[         cleanup |      MainThread] Waiting for instance to terminate.
[         cleanup |      MainThread] Done.
